In [1]:
# Import necessary libraries
from ultralytics import YOLO
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf
from PIL import Image as PILImage
import io
from IPython.display import Image, display
from PIL import Image as PILImage
import numpy as np
import tensorflow as tf
import io
import os
import pandas as pd
import torch
import json
from torch import float32, tensor
import torchmetrics
import cv2


2024-01-14 20:54:15.681398: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 20:54:15.681423: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 20:54:15.681933: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import os
import shutil

# Source directory containing images
src_dir = './TRAIN'

# Destination directory to copy images
dst_dir = './TEST'

# Check if destination directory exists, create it if it doesn't
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

# Loop through all files in source directory
for file_name in os.listdir(src_dir):
    # Check if file ends with .jpg
    if len(os.listdir(dst_dir)) == 50:
        break
    if file_name.endswith('.jpg'):
        # Construct full file paths
        src_path = os.path.join(src_dir, file_name)
        dst_path = os.path.join(dst_dir, file_name)
        # Copy file to destination directory
        shutil.copy(src_path, dst_path)




In [3]:
from transformers import TFViTForImageClassification

# Replace "your_fine_tuned_model_directory" with the actual directory where your fine-tuned model is stored.
model2 = TFViTForImageClassification.from_pretrained("./plantdoc_ViT_training/vit-base-patch16-224-in21k")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load the YOLO model
yolo_model = YOLO('./YOLOv8_training/multiclass_best.pt').to(device)

# Class mapping
# class_mapping = {'Apple Scab Leaf': 0, 'Apple leaf': 1, 'Apple rust leaf': 2, 'Bell_pepper leaf': 3, 'Bell_pepper leaf spot': 4, 'Blueberry leaf': 5, 'Cherry leaf': 6, 'Corn Gray leaf spot': 7, 'Corn leaf blight': 8, 'Corn rust leaf': 9, 'Peach leaf': 10, 'Potato leaf': 11, 'Potato leaf early blight': 12, 'Potato leaf late blight': 13, 'Raspberry leaf': 14, 'Soyabean leaf': 15, 'Squash Powdery mildew leaf': 16, 'Strawberry leaf': 17, 'Tomato Early blight leaf': 18, 'Tomato Septoria leaf spot': 19, 'Tomato leaf': 20, 'Tomato leaf bacterial spot': 21, 'Tomato leaf late blight': 22, 'Tomato leaf mosaic virus': 23, 'Tomato leaf yellow virus': 24, 'Tomato mold leaf': 25, 'Tomato two spotted spider mites leaf': 26, 'grape leaf': 27, 'grape leaf black rot': 28}

# Directory containing images to predict
directory = './datasets/test/images'
# directory = './TEST'

All model checkpoint layers were used when initializing TFViTForImageClassification.

All the layers of TFViTForImageClassification were initialized from the model checkpoint at ./plantdoc_ViT_training/vit-base-patch16-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTForImageClassification for predictions without further training.


In [4]:
class_mapping = {}

# Assuming your dataset is a pandas DataFrame named 'df' and the column with class labels is 'class'
for i in range(29):
    predicted_class_label = model2.config.id2label[i]
    class_mapping[predicted_class_label] = i

print("Class Mapping:", class_mapping)

Class Mapping: {'Soyabean leaf': 0, 'Tomato leaf late blight': 1, 'Corn Gray leaf spot': 2, 'Cherry leaf': 3, 'Tomato Septoria leaf spot': 4, 'Tomato two spotted spider mites leaf': 5, 'Potato leaf late blight': 6, 'Tomato leaf mosaic virus': 7, 'Corn rust leaf': 8, 'Squash Powdery mildew leaf': 9, 'Bell_pepper leaf': 10, 'Apple leaf': 11, 'Potato leaf early blight': 12, 'grape leaf black rot': 13, 'Strawberry leaf': 14, 'Apple rust leaf': 15, 'Tomato mold leaf': 16, 'Tomato leaf bacterial spot': 17, 'Corn leaf blight': 18, 'Blueberry leaf': 19, 'grape leaf': 20, 'Tomato leaf yellow virus': 21, 'Raspberry leaf': 22, 'Bell_pepper leaf spot': 23, 'Tomato Early blight leaf': 24, 'Potato leaf': 25, 'Peach leaf': 26, 'Tomato leaf': 27, 'Apple Scab Leaf': 28}


In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
preds=[]
# Define a list of dictionaries
data = []
# Create empty lists to store results
resized_cropped_images = []
predicted_classes = []

In [7]:
from transformers import ViTFeatureExtractor
import torch
import pandas as pd
import tensorflow as tf
from PIL import Image as PILImage
import os
import io
from torchvision.ops.boxes import box_iou

# Load the ViT feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('./plantdoc_ViT_training/vit-base-patch16-224-in21k')

# Load the YOLO results dataframe
df = pd.read_csv('./YOLOv8_training/new_labels.csv')

# Define the target size for ResNet50
target_size = (224, 224)

# Initialize arrays for predictions and targets
all_preds = []
all_targets = []

# IoU threshold for matching predictions with ground truth
iou_threshold = 0.5

# Process YOLO results
for filename in os.listdir(directory):
    filtered_df = df[df['filename'] == filename]
    results = yolo_model.predict(os.path.join(directory, filename))
    for x in results:
        for r in x:
            temp_preds = []
            temp_targets = []
            # Get the image and bounding box
            im_array = r.plot()  # plot a BGR numpy array of predictions
            im = PILImage.fromarray(im_array[..., ::-1])  # Convert BGR to RGB PIL image
            bbox = r.boxes.data[0, :4].cpu().numpy()  # Get the bounding box

            # Crop the detected object from the image
            cropped_im = im.crop((bbox[0], bbox[1], bbox[2], bbox[3]))

#             # Resize the cropped image to the target size
#             resized_cropped_im = cropped_im.resize(target_size)

#             # Save the resized and cropped image to a BytesIO object
#             img_io = io.BytesIO()
#             resized_cropped_im.save(img_io, 'PNG')

            # Append the resized and cropped image to the list (if needed)
            # resized_cropped_images.append(resized_cropped_im)

            # Extract features using ViT
            inputs = feature_extractor(images=cropped_im)
            pixel_values_tensor = tf.convert_to_tensor(inputs['pixel_values'], dtype=tf.float32)
            outputs = model2(pixel_values=pixel_values_tensor)
            logits = outputs.logits
            logits = tf.nn.softmax(logits)
            
            # model predicts one of the 1000 ImageNet classes
            predicted_class_index = tf.argmax(logits, axis=-1).numpy().item()
            predicted_class_label = model2.config.id2label[predicted_class_index]
            
            # Convert bounding box to tensor and move to GPU
            bbox_tensor = torch.tensor([[bbox[0], bbox[1], bbox[2], bbox[3]]]).to(device)
            
            # Convert labels to tensor and move to GPU
            label_tensor = torch.tensor([predicted_class_index]).to(device)

            # Convert scores to tensor and move to GPU
            score_tensor = torch.tensor([tf.reduce_max(logits).numpy()]).to(device) * r.boxes.conf

            # Append bounding box details to temp_preds
            temp_preds.append(dict(boxes=bbox_tensor, labels=label_tensor, scores=score_tensor))
            for index, row in filtered_df.iterrows():
                temp_targets.append({'boxes': torch.tensor([[row['xmin'], row['ymin'], row['xmax'], row['ymax']]]).to(device),
                                    'labels': torch.tensor([class_mapping[row['class']]]).to(device)})

            # Calculate IoU matrix between prediction and ground truth boxes
            iou_matrix = box_iou(torch.stack([r['boxes'].squeeze(0) for r in temp_preds]),
                                 torch.stack([t['boxes'].squeeze(0) for t in temp_targets]))

            # Find the index of the ground truth box with the highest IoU for each prediction
            max_iou_indices = torch.argmax(iou_matrix, dim=1)

            # Update temp_targets based on the highest IoU
            temp_targets = [temp_targets[i] for i in max_iou_indices]
            all_preds.extend(temp_preds)
            all_targets.extend(temp_targets)

/home/suryansh/anaconda3/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(



image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img2458.jpg: 480x640 7 Tomato leaf mosaic viruss, 58.6ms
Speed: 0.9ms preprocess, 58.6ms inference, 173.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img638.jpg: 576x640 1 Corn leaf blight, 56.3ms
Speed: 1.3ms preprocess, 56.3ms inference, 0.7ms postprocess per image at shape (1, 3, 576, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img454.jpg: 544x640 1 Apple Scab Leaf, 55.2ms
Speed: 1.5ms preprocess, 55.2ms inference, 0.6ms postprocess per image at shape (1, 3, 544, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1086.jpg: 416x640 1 grape leaf black rot, 55.0ms
Speed: 1.7ms preprocess, 55.0ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease

Speed: 1.7ms preprocess, 14.1ms inference, 1.3ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1171.jpg: 480x640 18 Tomato leafs, 9.8ms
Speed: 1.2ms preprocess, 9.8ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1538.jpg: 480x640 17 Tomato mold leafs, 10.1ms
Speed: 0.9ms preprocess, 10.1ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img38.jpg: 480x640 4 Tomato Early blight leafs, 5 Tomato leafs, 14.8ms
Speed: 1.8ms preprocess, 14.8ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img90.jpg: 640x480 1 Raspberry leaf, 1 Squash Powdery mildew leaf, 2 Tomato leafs, 5 Tomato leaf


image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1091.jpg: 480x640 11 Tomato leaf yellow viruss, 14.6ms
Speed: 1.6ms preprocess, 14.6ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img2512.jpg: 480x640 1 Bell_pepper leaf, 13.7ms
Speed: 1.7ms preprocess, 13.7ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img745.jpg: 640x416 1 Bell_pepper leaf, 53.3ms
Speed: 1.6ms preprocess, 53.3ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 416)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1965.jpg: 480x640 1 Tomato leaf bacterial spot, 14.2ms
Speed: 1.5ms preprocess, 14.2ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-

Speed: 1.6ms preprocess, 12.6ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img2406.jpg: 480x640 3 Apple Scab Leafs, 9.8ms
Speed: 0.9ms preprocess, 9.8ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img705.jpg: 640x640 3 Strawberry leafs, 11.5ms
Speed: 1.4ms preprocess, 11.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img119.jpg: 480x640 6 Tomato leafs, 9.7ms
Speed: 1.0ms preprocess, 9.7ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img942.jpg: 480x640 1 Corn leaf blight, 15.1ms
Speed: 1.3ms preprocess, 15.1ms inference, 1.3ms postprocess per image at shape

Speed: 1.6ms preprocess, 14.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 544)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1070.jpg: 480x640 6 Tomato Septoria leaf spots, 14.7ms
Speed: 1.3ms preprocess, 14.7ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img2118.jpg: 448x640 1 Apple leaf, 11.0ms
Speed: 1.6ms preprocess, 11.0ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img893.jpg: 512x640 1 Corn leaf blight, 12.5ms
Speed: 1.8ms preprocess, 12.5ms inference, 1.3ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1536.jpg: 640x608 1 Potato leaf, 3 Tomato Septoria leaf spots, 2 Tomato leaf yellow viruss, 13.6ms
Speed: 1.6m

Speed: 1.0ms preprocess, 6.4ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1695.jpg: 448x640 1 Corn rust leaf, 8.2ms
Speed: 0.9ms preprocess, 8.2ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1840.jpg: 480x640 5 Raspberry leafs, 9.9ms
Speed: 1.4ms preprocess, 9.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img976.jpg: 480x640 1 Corn rust leaf, 9.5ms
Speed: 0.9ms preprocess, 9.5ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1698.jpg: 480x640 1 Corn Gray leaf spot, 2 Corn leaf blights, 9.7ms
Speed: 1.3ms preprocess, 9.7ms inference, 0.5ms postproces

Speed: 1.5ms preprocess, 14.4ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1081.jpg: 416x640 1 Corn rust leaf, 11.7ms
Speed: 1.5ms preprocess, 11.7ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1919.jpg: 544x640 1 Apple leaf, 13.9ms
Speed: 1.7ms preprocess, 13.9ms inference, 1.3ms postprocess per image at shape (1, 3, 544, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img2419.jpg: 480x640 1 Apple Scab Leaf, 2 Apple leafs, 1 Apple rust leaf, 3 Peach leafs, 14.8ms
Speed: 1.5ms preprocess, 14.8ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img70.jpg: 384x640 1 Corn rust leaf, 11.5ms
Speed: 1.3ms preprocess, 11.5ms 


image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img272.jpg: 416x640 17 Peach leafs, 8.1ms
Speed: 1.1ms preprocess, 8.1ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img584.jpg: 640x544 1 Corn leaf blight, 14.1ms
Speed: 1.8ms preprocess, 14.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 544)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1138.jpg: 448x640 22 Soyabean leafs, 12.8ms
Speed: 1.6ms preprocess, 12.8ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1485.jpg: 480x640 1 Apple Scab Leaf, 14.9ms
Speed: 1.6ms preprocess, 14.9ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/te

In [8]:
# for img_data, predicted_class in zip(resized_cropped_images, predicted_classes):
#     display(img_data, predicted_class, class_mapping[predicted_class])


In [9]:
metrics = torchmetrics.detection.MeanAveragePrecision().to(device)
for i in range(len(all_preds)):
    print([all_preds[i]], [all_targets[i]])
metrics.update(all_preds, all_targets)
print(metrics.compute())

[{'boxes': tensor([[  0.0000, 160.5174, 208.9548, 284.2183]], device='cuda:0'), 'labels': tensor([7], device='cuda:0'), 'scores': tensor([0.4543], device='cuda:0')}] [{'boxes': tensor([[ 25, 160, 204, 279]], device='cuda:0'), 'labels': tensor([7], device='cuda:0')}]
[{'boxes': tensor([[428.7725,  90.2564, 565.3612, 248.9724]], device='cuda:0'), 'labels': tensor([21], device='cuda:0'), 'scores': tensor([0.1555], device='cuda:0')}] [{'boxes': tensor([[437,  83, 563, 237]], device='cuda:0'), 'labels': tensor([7], device='cuda:0')}]
[{'boxes': tensor([[130.5516,   0.0000, 266.3249, 218.1907]], device='cuda:0'), 'labels': tensor([7], device='cuda:0'), 'scores': tensor([0.2198], device='cuda:0')}] [{'boxes': tensor([[152,   7, 238, 183]], device='cuda:0'), 'labels': tensor([7], device='cuda:0')}]
[{'boxes': tensor([[  0.0000,  29.9494, 121.2516, 258.9439]], device='cuda:0'), 'labels': tensor([21], device='cuda:0'), 'scores': tensor([0.2383], device='cuda:0')}] [{'boxes': tensor([[ 25, 160, 2

[{'boxes': tensor([[763.2866, 382.9863, 865.3841, 482.4380]], device='cuda:0'), 'labels': tensor([21], device='cuda:0'), 'scores': tensor([0.5629], device='cuda:0')}] [{'boxes': tensor([[732, 380, 874, 485]], device='cuda:0'), 'labels': tensor([21], device='cuda:0')}]
[{'boxes': tensor([[725.7805, 609.4359, 789.0284, 706.9642]], device='cuda:0'), 'labels': tensor([21], device='cuda:0'), 'scores': tensor([0.5599], device='cuda:0')}] [{'boxes': tensor([[727, 560, 804, 723]], device='cuda:0'), 'labels': tensor([21], device='cuda:0')}]
[{'boxes': tensor([[450.0782, 219.9686, 536.3627, 288.3341]], device='cuda:0'), 'labels': tensor([21], device='cuda:0'), 'scores': tensor([0.5443], device='cuda:0')}] [{'boxes': tensor([[638, 245, 737, 350]], device='cuda:0'), 'labels': tensor([21], device='cuda:0')}]
[{'boxes': tensor([[ 45.9007, 180.8360, 160.8903, 368.1964]], device='cuda:0'), 'labels': tensor([7], device='cuda:0'), 'scores': tensor([0.1880], device='cuda:0')}] [{'boxes': tensor([[638, 24

[{'boxes': tensor([[651.9896, 488.2288, 790.7329, 701.8275]], device='cuda:0'), 'labels': tensor([19], device='cuda:0'), 'scores': tensor([0.7750], device='cuda:0')}] [{'boxes': tensor([[650, 491, 788, 706]], device='cuda:0'), 'labels': tensor([19], device='cuda:0')}]
[{'boxes': tensor([[1113.9841,  200.8782, 1503.5720,  441.3183]], device='cuda:0'), 'labels': tensor([19], device='cuda:0'), 'scores': tensor([0.7927], device='cuda:0')}] [{'boxes': tensor([[1138,  196, 1498,  447]], device='cuda:0'), 'labels': tensor([19], device='cuda:0')}]
[{'boxes': tensor([[369.0666, 185.9926, 708.6243, 354.5673]], device='cuda:0'), 'labels': tensor([19], device='cuda:0'), 'scores': tensor([0.7834], device='cuda:0')}] [{'boxes': tensor([[398, 183, 710, 344]], device='cuda:0'), 'labels': tensor([19], device='cuda:0')}]
[{'boxes': tensor([[ 927.5442,  717.6130, 1086.4841,  961.4435]], device='cuda:0'), 'labels': tensor([19], device='cuda:0'), 'scores': tensor([0.7898], device='cuda:0')}] [{'boxes': ten

[{'boxes': tensor([[230.7945, 401.7084, 276.7583, 442.5292]], device='cuda:0'), 'labels': tensor([19], device='cuda:0'), 'scores': tensor([0.2463], device='cuda:0')}] [{'boxes': tensor([[576, 214, 695, 303]], device='cuda:0'), 'labels': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[695.4197, 163.8000, 818.1473, 234.9998]], device='cuda:0'), 'labels': tensor([0], device='cuda:0'), 'scores': tensor([0.1077], device='cuda:0')}] [{'boxes': tensor([[576, 214, 695, 303]], device='cuda:0'), 'labels': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[ 96.5439,  75.9529, 334.5709, 389.4753]], device='cuda:0'), 'labels': tensor([28], device='cuda:0'), 'scores': tensor([0.1639], device='cuda:0')}] [{'boxes': tensor([[ 98,  77, 332, 384]], device='cuda:0'), 'labels': tensor([28], device='cuda:0')}]
[{'boxes': tensor([[  25.6659,    0.0000, 1458.8566, 1782.0000]], device='cuda:0'), 'labels': tensor([15], device='cuda:0'), 'scores': tensor([0.7362], device='cuda:0')}] [{'boxes': tensor([[  14,

{'map': tensor(0.3744), 'map_50': tensor(0.4589), 'map_75': tensor(0.4273), 'map_small': tensor(0.3281), 'map_medium': tensor(0.3583), 'map_large': tensor(0.3756), 'mar_1': tensor(0.4512), 'mar_10': tensor(0.4512), 'mar_100': tensor(0.4512), 'mar_small': tensor(0.4557), 'mar_medium': tensor(0.4433), 'mar_large': tensor(0.4468), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([ 0,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28], dtype=torch.int32)}
